In [1]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import TokenClassificationPipeline
from utils import PreProcess
import datasets
from torch.utils.data import DataLoader
import torch
from datetime import datetime
from IPython.display import HTML

c:\Users\zhma\Projects\deid\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wfumodel = './checkpoint-8600'
tokenizer = AutoTokenizer.from_pretrained(wfumodel)
tokenizer.model_max_length = 128
model = AutoModelForTokenClassification.from_pretrained(wfumodel)

In [3]:
wfu_dataset = datasets.load_dataset('wfudata', trust_remote_code=True)
all_text = wfu_dataset['test']['text']

In [4]:
len(all_text)

758

In [5]:
tokenizer = AutoTokenizer.from_pretrained(wfumodel)
tokenizer.model_max_length = 128
model = AutoModelForTokenClassification.from_pretrained(wfumodel)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
clf = TokenClassificationPipeline(model=model, tokenizer=tokenizer, device=0)

In [6]:
model.device

device(type='cuda', index=0)

In [7]:
print(datetime.now())
results = clf(all_text, stride=16, ignore_labels=['NORMAL'], aggregation_strategy='max', batch_size=16)
print(datetime.now())

2024-08-15 15:47:26.820220


c:\Users\zhma\Projects\deid\.venv\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


2024-08-15 15:48:29.843033


In [8]:
example_text = all_text[2]
example_result = results[2]

In [9]:
import sys
# del sys.modules['obfuscator']
from obfuscator import Obfuscator
obfuscator = Obfuscator()

In [10]:
example_result

[{'entity_group': 'IDNUM',
  'score': 0.99989796,
  'word': 's19 - 18004',
  'start': 21,
  'end': 30},
 {'entity_group': 'DATE',
  'score': 0.9998292,
  'word': '6 / 21 / 2019',
  'start': 41,
  'end': 50},
 {'entity_group': 'NAME',
  'score': 0.9998303,
  'word': 'edward hal kincaid',
  'start': 72,
  'end': 90},
 {'entity_group': 'NAME',
  'score': 0.9998336,
  'word': 'farmer, sheila dianne',
  'start': 111,
  'end': 132},
 {'entity_group': 'NAME',
  'score': 0.9998233,
  'word': 'o. a. hassan',
  'start': 388,
  'end': 399},
 {'entity_group': 'NAME',
  'score': 0.9998079,
  'word': 'javier ariel laurini',
  'start': 610,
  'end': 630},
 {'entity_group': 'DATE',
  'score': 0.9998233,
  'word': '6 / 26 / 2019 14 : 57 : 09',
  'start': 636,
  'end': 654},
 {'entity_group': 'NAME',
  'score': 0.9997913,
  'word': 'o. a. hassan',
  'start': 1079,
  'end': 1090}]

In [11]:
HTML(example_text)

In [12]:
HTML(obfuscator.parse_and_hide(example_text, example_result, second_shift=-86400))

In [13]:
example_text = all_text[3]
example_result = results[3]
example_result

[{'entity_group': 'HOSPITAL',
  'score': 0.99916697,
  'word': 'diabetes care center',
  'start': 0,
  'end': 20},
 {'entity_group': 'NAME',
  'score': 0.99957854,
  'word': 'ashlin mcguinn',
  'start': 22,
  'end': 36},
 {'entity_group': 'DATE',
  'score': 0.99979436,
  'word': '2 / 27 / 14',
  'start': 53,
  'end': 60},
 {'entity_group': 'NAME',
  'score': 0.9996427,
  'word': 'ashlin',
  'start': 313,
  'end': 319},
 {'entity_group': 'NAME',
  'score': 0.9996668,
  'word': 'ashlin',
  'start': 528,
  'end': 534},
 {'entity_group': 'NAME',
  'score': 0.99953794,
  'word': 'ashlin',
  'start': 607,
  'end': 613},
 {'entity_group': 'NAME',
  'score': 0.9995735,
  'word': 'ashlin',
  'start': 677,
  'end': 683},
 {'entity_group': 'NAME',
  'score': 0.99951994,
  'word': 'ashlin',
  'start': 718,
  'end': 724},
 {'entity_group': 'NAME',
  'score': 0.9995474,
  'word': 'ashlin',
  'start': 841,
  'end': 847},
 {'entity_group': 'NAME',
  'score': 0.99959296,
  'word': 'ashlin',
  'start': 

In [14]:
HTML(example_text)

In [15]:
HTML(obfuscator.parse_and_hide(example_text, example_result, second_shift=-86400))

In [16]:
outputs = []
for text, result in zip(all_text, results):
    outputs.append(obfuscator.parse_and_hide(text, result, -86400))

not parsed 6/30/2017 12/30/2016
not parsed 12/30/2017 12/30/2016
not parsed md10/23/2012 11:59:17
not parsed (march of 2017
not parsed (05/16/2018
not parsed (march of 2017
error in age -- {curr_span} ---
error in age -- {curr_span} ---
not parsed 4/21/2015 5/12/2015 5/26/2015
not parsed :  4/19/2016
not parsed 2002     2003
not parsed 1/12/2016 1/13/2016 3/4/2016 4/19/2016
not parsed 5/27/2015 1/12/2016 4/19/2016
not parsed :  3/7/2013
not parsed 2/7/2013 3/7/2013 3/28/2013
not parsed 17 13 15
not parsed 17 15 15
not parsed 10/12/2011 10/12/2010
not parsed 9/1/2015 1/22/2015
not parsed 1/1/2016 1/1/2006
not parsed 10/25/12electronic
not parsed (1/10/17
not parsed 09/09/13electronic
not parsed 05/24/19 1.
not parsed (06/2015
not parsed 1980 - 2015
not parsed 12/11/2017 1/23/2018 2/27/2018
not parsed md10/20/2012 06:59:24
not parsed 08/10/12          -7 / -7
not parsed 08/10/12          -7 / -7
error in age -- {curr_span} ---
not parsed 4/11/
not parsed oob-13:02
if-13:25
not parsed md1

In [17]:
HTML(outputs[2])

In [18]:
HTML(all_text[10])

In [19]:
HTML(outputs[10])

In [20]:
HTML(all_text[20])

In [21]:
HTML(outputs[20])

In [22]:
HTML(outputs[32])